In [4]:
#| default_exp vpg_buffer

In [16]:
#| export
import numpy as np
import torch
import vpg.vpg_core as core

In [2]:
#| export
class VPGBuffer:
    def __init__(self, obs_dim, act_dim, size, gamma, gae_lambda):
        self.obs_buffer = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.act_buffer = np.zeros(core.combined_shape(size, act_dim), dtype=np.float32)
        self.rew_buffer = np.zeros(size, dtype=np.float32)
        self.ret_buffer = np.zeros(size, dtype=np.float32)
        self.val_buffer = np.zeros(size, dtype=np.float32)
        self.adv_buffer = np.zeros(size, dtype=np.float32)
        self.log_prob_buffer = np.zeros(size, dtype=np.float32)
        
        self.gamma, self.gae_lambda = gamma, gae_lambda
        self.ptr, self.path_start_idx, self.max_size = 0, 0, size
    
    def store(self, obs, act, rew, val, log_prob):
        """
        Append one timestep of agent-environment interaction ot the buffer
        """
        
        assert self.ptr < self.max_size, "Buffer is filled up"
        self.obs_buffer[self.ptr] = obs
        self.act_buffer[self.ptr] = act
        self.rew_buffer[self.ptr] = rew
        self.val_buffer[self.ptr] = val
        self.log_prob_buffer[self.ptr] = log_prob
        self.ptr += 1
    
    def finish_path(self, last_val=0):
        """
        Call this at the end of a trajectory, or when a trajectory gets cut 
        off by an epoch ending. This looks back in the buffer to where the
        trajectory started, and uses rewards and value estimates from
        the whole trajectory to compute advantage estimates with GAE-Lambda,
        as well as compute the rewards-to-go for each state, to use as
        the targets for the value function.

        The "last_val" argument should be 0 if the trajectory ended
        because the agent reached a terminal state (died), and otherwise
        should be V(s_T), the value function estimated for the last state.
        This allows us to bootstrap the reward-to-go calculation to account
        for timesteps beyond the arbitrary episode horizon (or epoch cutoff).
        """
        
        # rewards and value estimates for this path
        path_slice = slice(self.path_start_idx, self.ptr)
        rews = np.append(self.rew_buffer[path_slice], last_val)
        vals = np.append(self.val_buffer[path_slice], last_val)
        
        # GAE implementation
        deltas = rews[:-1] + self.gamma*vals[1:] - vals[:-1]
        self.adv_buffer[path_slice] = core.discount_cumsum(deltas, self.gamma*self.gae_lambda)
        
        # Discounted reward-to-go
        self.ret_buffer[path_slice] = core.discount_cumsum(rews, self.gamma)[:-1]
        
        self.path_start_idx = self.ptr
    
    def get(self):
        """
        Call this at the end of an epoch to get all of the data from
        the buffer, with advantages appropriately normalized (shifted to have
        mean zero and std one). Also, resets some pointers in the buffer.
        
        Returns:
            (obs, act, ret, adv, log_prob)
        """
        
        assert self.ptr == self.max_size, "Buffer is not yet full"
        self.ptr, self.path_start_idx = 0, 0
        
        # normalize the advantage
        mean = np.mean(self.adv_buffer)
        std = np.std(self.adv_buffer)        
        self.adv_buffer = (self.adv_buffer - mean)/std
        
        res = dict(obs=self.obs_buffer, act=self.act_buffer, ret=self.ret_buffer,
                  adv=self.adv_buffer, logp=self.log_prob_buffer)
        return {k: torch.as_tensor(v, dtype=torch.float32) for k, v in res.items()}

In [3]:
#| hide

# import nbdev
# nbdev.nbdev_export()